In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from geospatial_4pointinpolygon_version import *

In [4]:
# Segment & Polygon intersection


# Function: create segments list (of points pair) based on points in polygon
def get_seg_from_pol(pol):
    pol_seg = []
    for i in range(1, pol.size):
        pol_seg.append(Segment(pol.points[i-1], pol.points[i]))
    #for i in range(0, (pol.size-1)):
    #    pol_seg.append(Segment(pol.points[i], pol.points[i+1]))
    # pol_seg.append(Segment(pol.points[pol.size-1], pol.points[0]))
    return pol_seg

# Function: calculate the intersection point of two segments which intersects
def get_intersect_point(seg1, seg2):
    A1 = seg1.end.y - seg1.start.y
    B1 = seg1.start.x - seg1.end.x
    C1 = seg1.end.x * seg1.start.y - seg1.start.x * seg1.end.y
    A2 = seg2.end.y - seg2.start.y
    B2 = seg2.start.x - seg2.end.x
    C2 = seg2.end.x * seg2.start.y - seg2.start.x * seg2.end.y
    if (A1 * B2 != A2 * B1):
        x = (C2 * B1 - C1 * B2)/(A1 * B2 - A2 * B1)
        y = (C1 * A2 - C2 * A1)/(A1 * B2 - A2 * B1)
    else:
        print("error: might need remove duplicated points first")
    return Point(x, y)

# Function: check if one segment touches the other if they already intersects, True is touch, False if intersection
def check_touch(seg1, seg2):
    if (seg1.start == seg2.start) or (seg1.start == seg2.end) or (seg1.end == seg2.start) or (seg1.end == seg2.end):
        # print("point touches")
        return True
    elif (seg1.start.x == seg1.end.x) and ((seg1.start.x == seg2.start.x) or (seg1.start.x == seg2.end.x)):
        # print("seg1 is vertical, and seg2 touches")
        return True
    elif (seg2.start.x == seg2.end.x) and ((seg2.start.x == seg1.start.x) or (seg2.start.x == seg1.end.x)):
        # print("seg2 is vertical, and seg1 touches")
        return True
    elif (seg1.start.x != seg1.end.x) and (seg2.start.x != seg2.end.x):
        f1 = (seg1.end.y - seg1.start.y)/(seg1.start.x - seg1.end.x)
        f2 = (seg2.end.y - seg2.start.y)/(seg2.start.x - seg2.end.x)
        if f1 == f2:
            # print("they are parallel")
            return True
        else:
            A1 = seg1.end.y - seg1.start.y
            B1 = seg1.start.x - seg1.end.x
            C1 = seg1.end.x * seg1.start.y - seg1.start.x * seg1.end.y
            A2 = seg2.end.y - seg2.start.y
            B2 = seg2.start.x - seg2.end.x
            C2 = seg2.end.x * seg2.start.y - seg2.start.x * seg2.end.y
            if (seg2.start.x*A1 + seg2.start.y*B1 + C1 == 0) or (seg2.end.x*A1 + seg2.end.y*B1 + C1 == 0) or (seg1.start.x*A2 + seg1.start.y*B2 + C2 == 0) or (seg1.end.x*A2 + seg1.end.y*B2 + C2 == 0):
                # print("one point lies on the other")
                return True
            else:
                # print("not touch, fine")
                return False
    else:
        # print("nope")
        return False
    
    
    
# Function: sorting points group
def sorting_PointGroup(pointgroup):
    n = len(pointgroup)
    xlist = []
    ylist = []
    for i in range(0, n):
        xlist.append(pointgroup[i].x)
        ylist.append(pointgroup[i].y)
    xy = pd.DataFrame()
    xy['x'] = xlist
    xy['y'] = ylist
    xy.sort_values(by=['x', 'y'], inplace = True)
    xy.reset_index(drop = True, inplace = True)
    pointgroup_sorted = []
    for j in range(0, n):
        pointgroup_sorted.append(Point(xy['x'][j], xy['y'][j]))
    # print("done")
    # print(pointgroup_sorted)
    return pointgroup_sorted


# The algorithm that works for concave hull also works for convex hull, 
# so it's not necessary to differeniate between convex and concave hull at the beginning.

# Function: Check if one segment intersects with one polygon or not and calculate the overlapping length
## Notice: Touching the edge of polygon is not considered as intersection in the following lines
def inter_seg_pol(seg, pol):
    # print(seg)
    # print(pol)
    
    # Create ordered segments from polygon
    segments_list = get_seg_from_pol(pol)
    # print(segments_list)
    # Prepare intersection segments list
    int_seg = []
    for i in range(0, (pol.size-1)):
        int_seg.append(seg.intersects(segments_list[i]))
    print(int_seg)
    ## The segment has no intersection with edges of the polygon
    if set(int_seg) == {False}:
        ### The two points of the segment are both outside the polygon - No Need for further calculation
        if (pol.containsPoint(seg.start) == False) and (pol.containsPoint(seg.end) == False):
            length = 0
            print(f"\nThe intersection length is: {length:.2f}")
            print("outside outside")
        ### The two points of the segment are both inside the polygon
        elif (pol.containsPoint(seg.start) == True) and (pol.containsPoint(seg.end) == True):
        #### Calculate the length of the segment
            length = seg.length
            print(f"\nThe intersection length is: {length:.2f}")
            print("inside inside")
        else:
            print("error: one point outside, one point inside and no intersection")
    ## The segment has intersection with edges of the polygon
    else:
        # print(int_seg)
        # print(len(int_seg))
        point_i = []
        length_i = []
        seg_i = []
        for i in range(0, len(int_seg)):
            # Find the edges which the segment intersects with
            # print(int_seg[i])
            # Generate a list of intersecting points
            # print("ok")
            # print(int_seg[i])
            if int_seg[i] == True:
                print("intersection!")
                # Check whether it is a touch or not
                # print(seg)
                # print(int_seg)
                # print(segments_list)
                print(segments_list[i])
                # print(segments_list[i+1])
                print(check_touch(seg, segments_list[i]))
                if check_touch(seg, segments_list[i]) == False:
                    # Find the intersecting point and store it in a list
                    # print(segments_list[i])
                    point_i.append(get_intersect_point(seg, segments_list[i]))
                    # print(point_i)
                # else:
                    # print("it's a touch instead of intersection")
            # else:
                # print("no intersection with this edge")
        # print(point_i)
        point_i.append(seg.start)
        point_i.append(seg.end)
        # print(point_i)    
        point_i = sorting_PointGroup(point_i)
        print(point_i)
        # for j in range(1, ((len(point_i))-1)):
        for j in range(1, len(point_i)):
            seg_i.append(Segment(point_i[j-1], point_i[j]))
            # Check if the center of the segments lie inside or outside the polygon
            x_m = (point_i[j-1].x + point_i[j].x)/2
            y_m = (point_i[j-1].y + point_i[j].y)/2
            p_i_m = Point(x_m, y_m)
            # print(p_i_m)
            # print(pol.containsPoint(p_i_m))
            if pol.containsPoint(p_i_m) == True:
                length_i.append((Segment(point_i[j-1], point_i[j])).length)
            else:
                length_i.append(0)
                # print('this small segment is outside this polygon')
        print(seg_i)
        print(length_i)
        length = sum(length_i)
        print(f"\nThe intersection length is: {length:.2f}")           
         
                
    
    
    
    
    
    #### Store all the intersecting point and create new segments
    #### Check if the the center point of the segment lie inside or outside the polygon for each segment
    #### Calculate the total length of the segments which its center lies inside the polygon
    
    



In [5]:
def inter_seg_pol_(seg, pol):
    # print(seg)
    # print(pol)
    
    # Create ordered segments from polygon
    segments_list = get_seg_from_pol(pol)
    # print(segments_list)
    # Prepare intersection segments list
    int_seg = []
    for i in range(0, (pol.size-1)):
        int_seg.append(seg.intersects(segments_list[i]))
    # print(int_seg)
    ## The segment has no intersection with edges of the polygon
    if set(int_seg) == {False}:
        ### The two points of the segment are both outside the polygon - No Need for further calculation
        if (pol.containsPoint(seg.start) == False) and (pol.containsPoint(seg.end) == False):
            length = 0
            # print(f"\nThe intersection length is: {length:.2f}")
            # print("outside outside")
        ### The two points of the segment are both inside the polygon
        elif (pol.containsPoint(seg.start) == True) and (pol.containsPoint(seg.end) == True):
        #### Calculate the length of the segment
            length = seg.length
            # print(f"\nThe intersection length is: {length:.2f}")
            # print("inside inside")
        else:
            length = length
            # print("error: one point outside, one point inside and no intersection")
    ## The segment has intersection with edges of the polygon
    else:
        # print(int_seg)
        # print(len(int_seg))
        length = 0
        point_i = []
        length_i = []
        seg_i = []
        for i in range(0, len(int_seg)):
            # Find the edges which the segment intersects with
            # print(int_seg[i])
            # Generate a list of intersecting points
            # print("ok")
            # print(int_seg[i])
            if int_seg[i] == True:
                # print("intersection!")
                # Check whether it is a touch or not
                # print(seg)
                # print(int_seg)
                # print(segments_list)
                # print(segments_list[i])
                # print(segments_list[i+1])
                # print(check_touch(seg, segments_list[i]))
                if check_touch(seg, segments_list[i]) == False:
                    # Find the intersecting point and store it in a list
                    # print(segments_list[i])
                    point_i.append(get_intersect_point(seg, segments_list[i]))
                    # print(point_i)
                # else:
                    # print("it's a touch instead of intersection")
            # else:
                # print("no intersection with this edge")
        # print(point_i)
        point_i.append(seg.start)
        point_i.append(seg.end)
        # print(point_i)    
        point_i = sorting_PointGroup(point_i)
        # print(point_i)
        # for j in range(1, ((len(point_i))-1)):
        for j in range(1, len(point_i)):
            seg_i.append(Segment(point_i[j-1], point_i[j]))
            # Check if the center of the segments lie inside or outside the polygon
            x_m = (point_i[j-1].x + point_i[j].x)/2
            y_m = (point_i[j-1].y + point_i[j].y)/2
            p_i_m = Point(x_m, y_m)
            # print(p_i_m)
            # print(pol.containsPoint(p_i_m))
            if pol.containsPoint(p_i_m) == True:
                length_i.append((Segment(point_i[j-1], point_i[j])).length)
            else:
                length_i.append(0)
                # print('this small segment is outside this polygon')
        # print(seg_i)
        # print(length_i)
        length = sum(length_i)
        return length
        # print(f"\nThe intersection length is: {length:.2f}")           


In [6]:
# Plot function for segments
def pS(seg):    
    x1 = [seg.start.x, seg.end.x]
    y1 = [seg.start.y, seg.end.y]
    plt.plot(x1, y1, linestyle="dashed", marker="o")
    plt.show()
def pSS(seg1, seg2):
    x1 = [seg1.start.x, seg1.end.x]
    y1 = [seg1.start.y, seg1.end.y]
    plt.plot(x1, y1, linestyle="dashed", marker="o")
    x2 = [seg2.start.x, seg2.end.x]
    y2 = [seg2.start.y, seg2.end.y]
    plt.plot(x2, y2, linestyle="dashed", marker="o")
    plt.show()
# Plot function for polygons
def pP(pol):
    xs = [i.x for i in pol]
    ys = [i.y for i in pol]
    plt.plot(xs, ys, linestyle='dashed')
    plt.show()

def plot_sp(seg, pol):
    xs = [i.x for i in pol]
    ys = [i.y for i in pol]
    plt.plot(xs, ys, linestyle='dashed')
    x1 = [seg.start.x, seg.end.x]
    y1 = [seg.start.y, seg.end.y]
    plt.plot(x1, y1, linestyle="dashed", marker="o")
    plt.show()

In [7]:
def inter_multi_seg_pol(seg_g, pol_g):
    n = len(seg_g)
    m = len(pol_g)
    inter_length = np.zeros((n,m))
    for i in range(0,n):
        for j in range(0,m):
            inter_length[i, j] = inter_seg_pol_(segs[i], pols[j])
    inter_length[np.isnan(inter_length)] = 0
    print(inter_length)
    sum_length = inter_length.sum()
    return sum_length